### IMPORTS

In [1]:
import ipywidgets
import cv2
from IPython.display import display
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
import traitlets
import time
import shutil
import os
import pandas as pd
import numpy as np

<br>

### HYPERPARAMETERS

In [2]:
# 小车控制的油门增益
STEERING_OFFSET = 0.035
THROTTLE_GAIN = 0.7

# 摄像头高度跟宽度
CAMERA_WIDTH = 448
CAMERA_HEIGHT = 336

# 帧率跟是否显示相机提要预览
FPS = 10
SHOW_CAMERA_PREVIEW = False

# 存储数据集的目录路径
#需換路徑 "datasets/"
DATASETS_DIR = "/home/greg/datasets/"
# 临时数据集的目录路径
TMP_DATASET_DIR = DATASETS_DIR + "tmp/"
# 注释文件的文件名
ANNOTATIONS_FILE = "annotations.csv"
# 临时数据集
TMP_ANNOTATIONS = TMP_DATASET_DIR + ANNOTATIONS_FILE

# 数据集的模式
DATASET_MODE = "training"

DATASET_NAME = "3"
# 主数据库目录路径
MAIN_DATASET_DIR = DATASETS_DIR + DATASET_NAME + "_" + DATASET_MODE + "/"
# 主要注释文件的路径
MAIN_ANNOTATIONS = MAIN_DATASET_DIR + ANNOTATIONS_FILE

<br>

### DATA COLLECTION

In [3]:
#该函数的作用是重置临时数据集目录
def reset_temp_dataset_dir():
    #判断临时数据集目录是否存在
    if not os.path.exists(TMP_DATASET_DIR):
        #如果不存在，则使用创建该目录
        os.makedirs(TMP_DATASET_DIR)
    else:
        #如果临时数据集目录已经存在，则先删除该目录及其所有内容
        shutil.rmtree(TMP_DATASET_DIR)
        #然后再次创建一个空的临时数据集目录
        os.makedirs(TMP_DATASET_DIR)

#通过调用reset_temp_dataset_dir()函数，执行重置临时数据集目录
reset_temp_dataset_dir()

#判断主数据集目录是否存在。如果主数据集目录不存在，则使用创建该目录
if not os.path.exists(MAIN_DATASET_DIR):
    os.makedirs(MAIN_DATASET_DIR)

In [4]:
# 函数用作于开始录制数据
def start_recording():
    # 用于重置临时数据集目录，以确保准备一个新的录制会话 
    reset_temp_dataset_dir()

   
# 用于保存录制的数据
def save_recording():
    for file in os.listdir(TMP_DATASET_DIR):
        if file.endswith('.csv'):
            # 检查主注释文件 MAIN_ANNOTATIONS 是否存在，且不是空的
            if os.path.exists(MAIN_ANNOTATIONS) and os.stat(MAIN_ANNOTATIONS).st_size > 0:
                # 打开主注释文件 MAIN_ANNOTATIONS
                with open(MAIN_ANNOTATIONS, 'a') as main:
                    # 打开时注释文件 TMP_ANNOTATIONS
                    with open(TMP_ANNOTATIONS) as tmp:
                        # 将临时注释文件中的每一行内容追加到主注释文件中
                        for line in tmp:
                            main.write(line)
                        # 关闭临时注释文件
                        tmp.close()
                    # 关闭主注释文件
                    main.close()
                #跳过将文件移动到主数据集目录
                continue
        #函数将文件从临时数据集目录移动到主数据集目录
        shutil.move(TMP_DATASET_DIR+file, MAIN_DATASET_DIR+file)
    #重置临时数据集目录，以准备进行下一次录制
    reset_temp_dataset_dir()

<br>

### CAR CONTROLLER

In [5]:
# 创建Nvidia Race car 类的实例car
car = NvidiaRacecar()
# 设置car的(油门增益)throttle_gain属性
car.throttle_gain = THROTTLE_GAIN
# 设置car的(方向盘偏移量)steering_offset属性
car.steering_offset = STEERING_OFFSET

In [ ]:
 # 创建控制器小部件
controller = ipywidgets.widgets.Controller(index=0)
# 显示控制器小部件
display(controller)
# 暂停1秒, 可改变暂时时间
time.sleep(1.0)

In [9]:
# 定义clamp function用于将值限制在给定的范围内
def clamp(value, val_min=-1.0, val_max=1.0):
    return min(val_max, max(val_min, value))

# 定义is_valid_press function，用于判断游戏手柄按钮event是否为有效的按下的event
def is_valid_press(x):
    return x['name'] == 'pressed' and x['new']

# Specific to Gamepad Mode 2 # 特定于游戏手柄模式2 

# 函数建立了与游戏手柄的连接，并创建控制器的方向盘轴值与car的转向属性之间的关联
steering_link = traitlets.dlink((controller.axes[2], 'value'), (car, 'steering'), transform=lambda x: clamp(-x))
# 函数建立了与游戏手柄的连接，并创建控制器的油门轴值与car的油门属性之间的关联
throttle_link = traitlets.dlink((controller.axes[1], 'value'), (car, 'throttle'), transform=lambda x: clamp(x))

# 函数建立了与游戏手柄的连接，并创建控制器的刹车按钮值与car的油门属性之间的关联
brake_throttle_link = traitlets.dlink((controller.buttons[7], 'value'), (car, 'throttle'), transform=lambda x: 0.0)
#函数建立了与游戏手柄的连接，并创建控制器的半油门按钮值与car的油门属性之间的关联
half_throttle_link = traitlets.dlink((controller.buttons[5], 'value'), (car, 'throttle'), transform=lambda x: -0.5 if x > 0.5 else 0)

# 当控制器上的特定按钮被按下时调用start_recording() function开始录制
controller.buttons[2].observe(lambda x: start_recording() if is_valid_press(x) else ())
# 当控制器上的特定按钮被按下时调用save_recording() function开始保存录制内容
controller.buttons[1].observe(lambda x: save_recording() if is_valid_press(x) else ())

<br>

### CAMERA

In [10]:
# 创建了一个 CSICamera 类的实例 camera，用于获取摄像头图像
camera = CSICamera(width=CAMERA_WIDTH, height=CAMERA_HEIGHT, capture_fps=FPS)
# 启动摄像头
camera.running = True

In [11]:
# 定义函数以保存带有注释的相机帧
def save_annotated_camera_frame(frame):
    timestamp = str(int(time.time()*1000))
    encoded_image = cv2.imencode('.jpg', frame)[1]
    
    # 保存相机帧的函数
    def save_camera_frame():
        cv2.imwrite(TMP_DATASET_DIR+timestamp+".jpg", frame)

    # 保存时间戳、汽车的转向和油门值以特定的格式的函数    
    def save_annotation():
        with open(TMP_ANNOTATIONS, 'a') as f: 
            f.write(timestamp + ", " + str(round(car.steering, 2)) + ", " + str(round(car.throttle, 2)) + "\n")
            f.close()
    
    # 调用保存相机帧到临时数据集目录
    save_camera_frame()
    # 调用保存注释到临时注释文件
    save_annotation()
    
    #将帧图像编码为 JPEG 格式的图像
    encoded_image = cv2.imencode('.jpg',frame)[1]
    #将图像数据转换为字节类型然后返回
    return bytes(encoded_image)

In [12]:
#创建一个 ipywidgets.Image 对象，并设置图像的格式为 JPEG
image = ipywidgets.Image(format='jpeg')
#如果调用到SHOW_CAMERA_PREVIEW 就会将图像显示出来
if SHOW_CAMERA_PREVIEW:
    display(image)

In [13]:
# 创建相机和图像之间的关联，并通过save_annotated_camera_frame函数进行转换和保存
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=save_annotated_camera_frame)